# Getting numeric financial information from XBRL data - Part I

This notebook contains example Python code to get numeric data available in XBRL financial reports using the [SEC's EDGAR XBRL API](https://www.sec.gov/edgar/sec-api-documentation).

**Made by:**  [Roman Chychyla](https://people.miami.edu/profile/rxc303@miami.edu)

##  EDGAR Application Programming Interfaces - Background
The SEC has recently created a web-based EDGAR Application Programming Interface (API). It allows users to obtain corporate financial XBRL data in JSON (JavaScript Object Notation) format.

### What is JSON format?
JSON format is a file format for data interchange (and is similar to XML format used to file XBRL data). It is lightweight and human-readable. It uses a set of brackets, commas, quotes, and other characters to represent data in a text file.

For example, basic information about a filer can represented in a JSON text file as follows:

**Microsoft.json**
```json
{"cik":"789019",
  "name":"MICROSOFT CORP",
  "sic":"7372",
  "sicDescription":"Services-Prepackaged Software",
  "tickers":["MSFT"],
  "exchanges":["Nasdaq"],
  "ein":"911144442",
  "category":"Large accelerated filer",
  "fiscalYearEnd":"0630",
  "stateOfIncorporation":"WA",
  "address":
    {"mailing":
      {"street1":"ONE MICROSOFT WAY",
       "street2":null,"city":"REDMOND",
       "stateOrCountry":"WA",
       "zipCode":"98052-6399",
       "stateOrCountryDescription":"WA"},
     "phone":"425-882-8080"}
}
```

### How do I access JSON data in EDGAR API?
The EDGAR API's JSON data can be accessed through HTTP (web) requests. These requests can be generated by a web browser or by any programming language that supports HTTP requests.

The data is organized at different levels:

*Filer-level request*:

[https://data.sec.gov/api/xbrl/companyfacts/CIK0000789019.json](https://data.sec.gov/api/xbrl/companyfacts/CIK0000789019.json) - all facts for the filer with the CIK of 0000789019.

*Filer-concept level request*:

[https://data.sec.gov/api/xbrl/companyconcept/CIK0000789019/us-gaap/AccountsPayableCurrent.json](https://data.sec.gov/api/xbrl/companyconcept/CIK0000789019/us-gaap/AccountsPayableCurrent.json) - all Current Account Payable facts for the filer with the CIK of 0000789019.

*Fact-period level request*:

[https://data.sec.gov/api/xbrl/frames/us-gaap/NetIncomeLoss/USD/CY2019.json](https://data.sec.gov/api/xbrl/frames/us-gaap/NetIncomeLoss/USD/CY2019.json) -  all Net Income (Loss) facts for the (calendar) year 2019.

[https://data.sec.gov/api/xbrl/frames/us-gaap/NetIncomeLoss/USD/CY2019Q2.json](https://data.sec.gov/api/xbrl/frames/us-gaap/NetIncomeLoss/USD/CY2019Q2.json) -  all Net Income (Loss) facts for the second quarter of 2019.

[https://data.sec.gov/api/xbrl/frames/us-gaap/AccountsPayableCurrent/USD/CY2019Q1I.json](https://data.sec.gov/api/xbrl/frames/us-gaap/AccountsPayableCurrent/USD/CY2019Q1I.json) - all Current Account Payable (instantaneous) facts as of the first quarter of 2019.

The EDGAR API data can be also downloaded in bulk (at the filer-level). This is the most optimal choice when working with the population of XBRL data:

(https://www.sec.gov/Archives/edgar/daily-index/xbrl/companyfacts.zip)[https://www.sec.gov/Archives/edgar/daily-index/xbrl/companyfacts.zip]


### What type of XBRL data is available through EDGAR API?
The API presents data at an easy-to-understand and simple-to-process level at the cost of reporting only **numeric**, **US GAAP**, **non-extension** XBRL facts. In other words, textual data (such as footnote text) or company-specific XBRL data items are not included in the API.

## Example: Getting numeric XBRL data through EDGAR API for a single company using Python 

We will first demonstrate how to extract numeric XBRL data using EDGAR API for a single company, and later extend the example to multiple filers.

Folder `companyfacts` contains ten random EDGAR API's filer-level JSON files. We will use the files in this folder in this example.

In [ ]:
# python library to work with files and folders paths (locations)
from pathlib import Path

# specify path to folder with JSON files
input_xbrl_data_folder =  Path('./companyfacts')

# pick the first file in the folder
file = input_xbrl_data_folder / 'CIK0001271046.json'

In [ ]:
# python library to work with JSON data
import json

# open the json file and use Python's JSON parser to parse it (into a Python dictionary)
with open(file,'r') as f:
    json_data = json.load(f)

# output the contents of the parsed JSON object
# the output is very long; if using Jupyter Notebook or Jupyter Lab, right-click on the output cell and select "Enable Scrolling for Outputs".
json_data

We can access JSON values in `json_data` object. For example, let us extract CIK information.

In [ ]:
cik = json_data['cik']

# display the CIK number
cik

EDGAR API's JSON files store US GAAP XBRL facts in `us-gaap` node that is a part of `facts` JSON node.

In [ ]:
# US GAAP tags are located under facts -> us-gaap
us_gaap_facts = json_data['facts']['us-gaap']
us_gaap_facts

To access values of a specific tag (concept), we need to indicate the tag's name and the units of measurement (e.g., currency). For example, to list all values of Total Assets for the filer, we can use the following code:

In [ ]:
total_assets_facts = us_gaap_facts['Assets']['units']['USD']
total_assets_facts

 While it is possible to read and understand raw JSON data, we can improve the presentation by convert JSON data to the tabular format:

In [ ]:
# library to work with tabular data
import pandas as pd

# convert total assets facts to a dataframe (table)
total_assets_df = pd.DataFrame(total_assets_facts)
# display the first ten records of the dataframe
total_assets_df.head(10)

The resulting table has a number of duplicate values because the same numerica data were reported in multiple filings. We can keep only unique data by keeping the most recent data points for a given fiscal period.

In [ ]:
# sort the values by the period end date, and for a given fiscal year and period type, keep the most recent XBRL facts.
# replace the original table with the resulting table
total_assets_dof = total_assets_df.sort_values('end',ascending=False).drop_duplicates(['fy','fp'])
# display the first ten records of the new dataframe
total_assets_df.head(10)

We can add values to the existing dataframe. For example:

In [ ]:
# add CIK number
total_assets_df['cik'] = cik
# add tag name
total_assets_df['tag'] = 'Assets'

# display the first ten records of the dataframe
total_assets_df.head(10)

To extract all possible XBRL facts from a company's JSON file, we simply need to repeat all of the above steps for all XBRL tags present in the JSON data:

In [ ]:
# create an empty list; it will be used to store dataframes for all XBRL tags present in the JSON data
tag_dfs = []

# loop over all XBRL tags, and process them one by one
for tag,details in json_data['facts']['us-gaap'].items():
    # we will only consider monetary XBRL tags measures in U.S. Dollars
     if 'units' in details and 'USD' in details['units']:
        # get all facts for the given tag
        tag_facts = details['units']['USD']
        # create dataframe
        tag_df = pd.DataFrame(tag_facts)
        # remove  duplicates
        tag_df =  tag_df.sort_values('end',ascending=False).drop_duplicates(['fy','fp'])
        # add CIK information
        tag_df['cik'] = cik
        # add tag name information
        tag_df['tag'] = tag
        # add the final table to the list of tables
        tag_dfs.append(tag_df)

Finally, we create a union of all the datasets in `tag_dfs` list that will result in a one big table with financial XBRL data.

In [ ]:
# merge (vertically) alll the datasets
# the last part - reset_index(drop=True) - is optional; it simply resets index numbering
cik_df = pd.concat(tag_dfs).reset_index(drop=True)
# output the result
cik_df

### Keeping only annual data items

The above dataset contains both annual and quarterly data. If we are interested in the annual data only, we can filter the data items based on the value of fiscal period variable, `fp`.

In [ ]:
# keep only annual data items
annual_cik_df = cik_df[cik_df['fp'] == 'FY'].reset_index(drop=True)
annual_cik_df

### Converting long data to wide

The table above has observations at the *company-period-tag* level (*long table* format). In most cases, we prefer wide table format where observations are at the *company-period* level with each XBRL tag having its own column (especially when we have observations for multiple companies). Therefore, in the next step, we will convert data to the *wide table* format.

In [ ]:
# convert dataframe to the wide dataframe at the CIK, fiscal year, and fiscal period level
annual_cik_df_wide = pd.pivot(annual_cik_df, index = ['cik','fy','fp'], columns = 'tag', values = 'val').reset_index()
# display the wide dataframe
annual_cik_df_wide.head()